<a href="https://colab.research.google.com/github/AlejoBI/IO/blob/main/Taller_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalamos la librería / software de optimización de Google
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.2

In [2]:
#Importar el módulo de PL de G-OR
from ortools.linear_solver import pywraplp

#Elegir el solucionador
# param ["GLOP solver para modelos continuos", "SCIP solver para modelos de programacion continua, entrea y mixta"]
solucionador =  'SCIP'
solver = pywraplp.Solver.CreateSolver(solucionador)

In [19]:
peso_max = 15
volumen_max = 15000
peso = [5, 7, 5, 2, 4, 6]
volumen = [4500, 4500, 14000, 1000, 2250, 5000]

items = [0, 1, 2, 3, 4, 5]
vehiculos = [0, 1, 2, 3, 4, 5]

#NumVar para continua
#IntVar para entera
#BoolVar para boolean

x = {}

for i in items:
  for j in vehiculos:
    x[i, j] = solver.IntVar(0, 1, '')

y = {}

for j in vehiculos:
  y[j] = solver.IntVar(0, 1, '')

#Restricciones
for i in items:
  solver.Add(sum(x[i, j] for j in vehiculos) == 1)

for j in vehiculos:
  solver.Add(sum(x[i, j] * peso[i] for i in items) <= peso_max * y[j])
  solver.Add(sum(x[i, j] * volumen[i] for i in items) <= volumen_max * y[j])

#Funcion objetivo (Maximize o Minimize)
solver.Minimize(sum(y[j] for j in vehiculos))

#Invocando solución
status = solver.Solve()

for i in items:
  for j in vehiculos:
    if x[i, j].solution_value() == 1: #Invoca las variable sy las compara
      print(f"La carga {i} se transporta en el vehiculo {j}")

print(f"Numero minimo de vehiculos: {solver.Objective().Value()}") #Solucion


La carga 0 se transporta en el vehiculo 2
La carga 1 se transporta en el vehiculo 2
La carga 2 se transporta en el vehiculo 1
La carga 3 se transporta en el vehiculo 0
La carga 4 se transporta en el vehiculo 0
La carga 5 se transporta en el vehiculo 0
Numero minimo de vehiculos: 3.0


In [20]:
for j in vehiculos:
  peso_total_vehiculo = sum(x[i, j].solution_value() * peso[i] for i in items)
  volumen_total_vehiculo = sum(x[i, j].solution_value() * volumen[i] for i in items)
  if y[j].solution_value() == 1:
    print(f"El vehiculo {j} tiene un peso total de {peso_total_vehiculo} y un volumen total de {volumen_total_vehiculo}")


El vehiculo 0 tiene un peso total de 12.0 y un volumen total de 8250.0
El vehiculo 1 tiene un peso total de 5.0 y un volumen total de 14000.0
El vehiculo 2 tiene un peso total de 12.0 y un volumen total de 9000.0
